You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Importance of staying updated on the latest trends and news in AI

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements
- Natural language processing innovations
- AI in healthcare and medical research
- Ethical considerations in AI development

2. Key Players in the AI Industry:
- Google's DeepMind
- 

I now can give a great answer

Final Answer: 

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has revolutionized various industries by providing advanced technologies and solutions. From machine learning to natural language processing, AI has transformed the way businesses operate and innovate. Staying updated on the latest trends and news in AI is crucial for individuals and organizations looking to leverage these technologies effectively.

## Latest Trends in Artificial Intelligence
In 2021, AI continues to advance rapidly, with significant progress in machine learning algorithms and models. Natural language processing innovations have also gained traction, enabling AI systems to understand and generate human language more accurately. The integration of AI in healthcare and medical research has shown promising results, enhancing diagnostics, treatment planning, and patient care. However, ethical considerations in AI de

- Display the results of your execution as markdown in the notebook.

In [15]:
from IPython.display import Markdown
Markdown(result)

# The Importance of Mental Health in the Workplace

In recent years, there has been a significant emphasis on mental health in the workplace, particularly in light of the challenges posed by the COVID-19 pandemic. Employers, HR departments, mental health organizations, and advocates have all come to realize the importance of prioritizing mental well-being in the workplace. Research has shown that mental health directly influences productivity and employee well-being, making it a critical component of overall workplace culture.

## Statistics on Mental Health Issues in the Workplace

The World Health Organization highlights that mental health conditions such as depression and anxiety are key contributors to absenteeism and presenteeism in work settings. Shockingly, the WHO estimates that depression and anxiety disorders result in a staggering $1 trillion loss in global productivity annually. Furthermore, the American Psychological Association notes that stress-related concerns are prevalent in workplaces, leading to reduced job satisfaction and heightened turnover rates.

## Strategies for Promoting Mental Health at Work

To tackle mental health challenges in the workplace, employers can adopt a range of strategies to enhance the well-being of their employees. These strategies may involve providing access to mental health services, offering flexible work arrangements, and fostering a supportive work environment that destigmatizes mental health issues. Training initiatives focusing on mental health awareness and resilience building can also be instrumental in empowering employees to effectively manage stress and maintain a healthy work-life balance.

## Benefits of Supporting Mental Health in the Workplace

Supporting mental health in the workplace not only benefits employees but also contributes to the overall productivity and success of the organization. Employees who feel supported in their mental well-being tend to be more engaged, motivated, and efficient. They are also less likely to experience burnout and are more inclined to remain with the company in the long run. By prioritizing mental health in workplace policies and practices, employers can establish a positive and inclusive work environment where employees feel valued and well-supported.

In conclusion, the significance of mental health in the workplace cannot be overstated. Employers play a pivotal role in championing mental well-being among their employees, ultimately resulting in heightened productivity, job satisfaction, and overall prosperity for the organization. It is imperative for employers to prioritize mental health in their workplace policies and practices to cultivate a nurturing and thriving work environment for all employees.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [16]:
topic = "Carbon Sequestration and Carbon Capture"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Carbon Sequestration and Carbon Capture.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan Document: 
Title: The Importance of Carbon Sequestration and Carbon Capture

Outline:
I. Introduction
    A. Definition of Carbon Sequestration and Carbon Capture
    B. Importance of these processes in mitigating climate change
II. Latest Trends in Carbon Sequestration and Carbon Capture
    A. Advancements in technology and research
    B. Global initiatives and policies promoting carbon sequestration
III. Key Players in the Industry
    A. Leading companies investing in carbon c

I now can give a great answer

Final Answer:
# The Importance of Carbon Sequestration and Carbon Capture

## Introduction
Carbon sequestration and carbon capture are essential processes in the fight against climate change. **Carbon sequestration** refers to the natural or artificial process by which carbon dioxide is removed from the atmosphere and stored in carbon sinks such as forests, oceans, and soil. On the other hand, **carbon capture** involves capturing carbon dioxide emissions from sources like power plants and industrial facilities before they are released into the atmosphere. These processes play a crucial role in reducing greenhouse gas emissions and mitigating the effects of global warming.

## Latest Trends in Carbon Sequestration and Carbon Capture
Recent advancements in technology and research have significantly improved the efficiency and effectiveness of carbon sequestration and carbon capture methods. Innovations such as direct air capture and enhanced weathering tec

In [17]:
Markdown(result)

# The Importance of Carbon Sequestration and Carbon Capture

## Introduction
Carbon sequestration and carbon capture are essential processes in the fight against climate change. **Carbon sequestration** refers to the natural or artificial process by which carbon dioxide is removed from the atmosphere and stored in carbon sinks such as forests, oceans, and soil. On the other hand, **carbon capture** involves capturing carbon dioxide emissions from sources like power plants and industrial facilities before they are released into the atmosphere. These processes play a crucial role in reducing greenhouse gas emissions and mitigating the effects of global warming.

## Latest Trends in Carbon Sequestration and Carbon Capture
Recent advancements in technology and research have significantly improved the efficiency and effectiveness of carbon sequestration and carbon capture methods. Innovations such as direct air capture and enhanced weathering techniques are making it possible to remove larger quantities of carbon dioxide from the atmosphere. Moreover, global initiatives and policies, such as the Paris Agreement, are driving countries to invest in carbon sequestration projects and implement carbon capture technologies to meet their emissions reduction goals.

## Key Players in the Industry
Leading companies like Carbon Engineering and Climeworks are at the forefront of investing in carbon capture technology, developing innovative solutions to tackle climate change. Organizations such as The Nature Conservancy and Terra Global are actively involved in carbon sequestration projects, working to restore degraded ecosystems and enhance natural carbon sinks. These key players are driving progress towards a more sustainable future by promoting the adoption of carbon sequestration and carbon capture technologies.

## Target Audience Analysis
Environmental enthusiasts, businesses looking to reduce their carbon footprint, and policy makers are the primary audience for information on carbon sequestration and carbon capture. Environmental enthusiasts are concerned about the impact of climate change and are eager to support initiatives that promote sustainable practices. Businesses are increasingly recognizing the importance of reducing their carbon emissions and are seeking innovative solutions to achieve carbon neutrality. Policy makers play a crucial role in implementing regulations and policies that support carbon sequestration and carbon capture efforts.

## Pain Points of the Audience
The audience's main pain points revolve around concerns about the escalating impact of climate change and the urgent need to implement sustainable practices. Environmental enthusiasts are alarmed by the rising global temperatures, extreme weather events, and biodiversity loss attributed to climate change. Businesses are facing pressure from consumers and stakeholders to reduce their carbon footprint and adopt environmentally friendly practices. Policy makers are under scrutiny to enact policies that effectively address climate change and promote sustainable development.

## Call to Action
As individuals, we can support carbon sequestration projects by participating in reforestation efforts, reducing our energy consumption, and advocating for sustainable practices in our communities. Businesses can reduce their carbon emissions by investing in renewable energy sources, implementing energy-efficient technologies, and offsetting their remaining emissions through carbon offset programs. Policy makers can support carbon sequestration and carbon capture initiatives by enacting legislation that incentivizes emission reductions and promotes the development of carbon-neutral technologies.

In conclusion, carbon sequestration and carbon capture are vital tools in the fight against climate change. By understanding the importance of these processes, supporting initiatives that promote sustainable practices, and advocating for policies that prioritize emissions reduction, we can collectively work towards a more sustainable future for our planet.

### Resources:
- IPCC Special Report on Global Warming of 1.5°C
- Carbon Capture and Storage Association
- United Nations Framework Convention on Climate Change

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).